<a href="https://colab.research.google.com/github/tom00ti/HTMLCSS_practice/blob/master/SanMoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ライブラリの読み込み

In [1]:
import numpy as np

シミュレータクラス

In [3]:
#シミュレータクラス
class MySimulator():
    def __init__(self):
        self.reset()
#初期化
    def reset(self):
        self._state = np.zeros(9, dtype=np.int32)#マスの設定
        return self._state
#行動による状態変化
    def step(self, action, turn):
        rewards = [0,0]
        done = False
        if self._state[action] != 0:#すでに〇か×が書かれているところに打った場合
            done = True
            rewards[turn] = -1#打ったエージェントだけマイナスの報酬
            return self._state, rewards, done
        self._state[action] = turn+1
        #3つ並んだかを判定
        ptn = [[0,1,2],[3,4,5],[6,7,8],[0,3,6],[1,4,7],[2,5,8],[0,4,8],[2,4,6]]
        for i in range(8):
            if self._state[ptn[i][0]] == turn+1 and \
               self._state[ptn[i][1]] == turn+1 and \
               self._state[ptn[i][2]] == turn+1:
                rewards[turn] = 1#勝ったエージェントにプラスの報酬
                rewards[(turn+1)%2] = -1#負けたエージェントにマイナスの報酬
                done = True
                return self._state, rewards, done            
        return self._state, rewards, done
    def show_Board(self):
        mb = {0:' ', 1:'O', 2:'X'}
        i=0
        print(mb[self._state[i*3]], "|", mb[self._state[i*3+1]], "|", mb[self._state[i*3+2]] )
        print("----------" )
        i=1
        print(mb[self._state[i*3]], "|", mb[self._state[i*3+1]], "|", mb[self._state[i*3+2]] )
        print("----------" )
        i=2
        print(mb[self._state[i*3]], "|", mb[self._state[i*3+1]], "|", mb[self._state[i*3+2]] )
    

Q値クラス

In [ ]:
#Q値クラスの設定
class MyQTable():
    def __init__(self, train=True):
        if train:
            print("Training")
            QV0=np.zeros((3**9,9), dtype=np.float32)
            QV1=np.zeros((3**9,9), dtype=np.float32)
        else:
            print("Game Start")
            QV0=np.zeros((3**9,9), dtype=np.float32)
            QV1 = np.loadtxt('Q1value.txt')
        self._QVs = [QV0, QV1]
#行動の選択
    def get_action(self, state, epsilon, turn):
        qv= self._QVs[turn]
        s = 0
        for i in range(9):
            s = s + state[i]*(3**i)
        if epsilon > np.random.uniform(0, 1):#徐々に最適行動のみをとる、ε-greedy法
            next_action = np.random.choice(range(9))
        else:
            a = np.where(qv[s]==qv[s].max())[0]
            next_action = np.random.choice(a)
        return next_action
    def update_Qtable(self, act, reward, state, state_old, turn):
        qv= self._QVs[turn]
        s = 0
        so = 0
        for i in range(9):
            s = s + state[i]*(3**i)
            so = so + state_old[i]*(3**i)
        alpha = 0.5
        gamma = 0.9
        maxQ = np.max(qv[s])
        qv[so,act] = (1-alpha)*qv[so,act]+alpha*(reward + gamma*maxQ);
        self._QVs[turn] = qv
    def save_Qtable(self):
        np.savetxt('Q0value.txt', self._QVs[0])
        np.savetxt('Q1value.txt', self._QVs[1])

学習の実行

In [ ]:
def main():
    num_episodes = 100000  #総試行回数
    env = MySimulator()
    tab = MyQTable()
    wins = [0,0,0,0,0]
    for episode in range(num_episodes):  #試行数分繰り返す
        if episode%10000==0:
            wins = [0,0,0,0,0]
        state = env.reset()
        state_old = [state,state]
        rewards = [0,0]
        actions = [0,0]
        epsilon = (1 / (episode + 1))+0.1
        step = 0
        while(1):
            actions[0] = tab.get_action(state, epsilon, 0)
            state_old[0] = np.copy(state)
            state, rewards, done = env.step(actions[0], 0)
            tab.update_Qtable(actions[1], rewards[1], state, state_old[1], 1)
            if done==True:
                tab.update_Qtable(actions[0], rewards[0], state, state_old[0], 0)
                if rewards[0]==-1:
                    wins[1]+=1
                else:
                    wins[0]+=1
                break
            step +=1
            if step==9:
                break
            actions[1] = tab.get_action(state, epsilon, 1)
            state_old[1] = np.copy(state)
            state, rewards, done = env.step(actions[1], 1)
            tab.update_Qtable(actions[0], rewards[0], state, state_old[0], 0)
            if done==True:
                tab.update_Qtable(actions[1], rewards[1], state, state_old[1], 1)
                if rewards[0]==-1:
                    wins[3]+=1
                else:
                    wins[2]+=1
                break
            step +=1
        if step==9:
            wins[4]+=1
        if (episode+1)%10000==0:
            print(wins,sum(wins))

    tab.save_Qtable()
    print("Training Finish.")
  
if __name__ == '__main__':
    main()

Training
[1117, 3309, 3089, 460, 2025] 10000
[1443, 2230, 1998, 519, 3810] 10000
[1491, 2110, 1917, 566, 3916] 10000
[1468, 2014, 1884, 496, 4138] 10000
[1402, 1895, 1703, 458, 4542] 10000
[1555, 1906, 1677, 450, 4412] 10000
[1446, 1875, 1788, 526, 4365] 10000
[1624, 1783, 1559, 486, 4548] 10000
[1474, 1821, 1688, 500, 4517] 10000
[1595, 1743, 1481, 471, 4710] 10000
Training Finish.


対戦の実行

In [ ]:
def show_InitBoard():
    i=0
    print(i*3, "|", i*3+1, "|", i*3+2 )
    print("----------" )
    i=1
    print(i*3, "|", i*3+1, "|", i*3+2 )
    print("----------" )
    i=2
    print(i*3, "|", i*3+1, "|", i*3+2 )
def main_play():
    show_InitBoard()
    num_episodes = 1  #総試行回数
    env = MySimulator()
    tab = MyQTable(False)

    state = env.reset()
    state_old = [state,state]
    rewards = [0,0]
    actions = [0,0]
    step = 0
    while(1):
        actions[0] = int(input('[0-8]'))
        state, rewards, done = env.step(actions[0], 0)
        env.show_Board()
        if done==True:
            if rewards[0]==-1:
                print('Penalty. You lose.')
            else:
                print('You win!!!')
            break
        step +=1
        if step==9:
            break
        actions[1] = tab.get_action(state, 0, turn=1)
        print("Agent action:", actions[1])
        state, rewards, done = env.step(actions[1], 1)
        env.show_Board()
        if done==True:
            if rewards[1]==-1:
                print('Penalty. You win.')
            else:
                print('You loose.')
            break
        step +=1
  
if __name__ == '__main__':
    main_play()


0 | 1 | 2
----------
3 | 4 | 5
----------
6 | 7 | 8
Game Start
[0-8]1
  | O |  
----------
  |   |  
----------
  |   |  
Agent action: 0
X | O |  
----------
  |   |  
----------
  |   |  
[0-8]2
X | O | O
----------
  |   |  
----------
  |   |  
Agent action: 4
X | O | O
----------
  | X |  
----------
  |   |  
[0-8]3
X | O | O
----------
O | X |  
----------
  |   |  
Agent action: 8
X | O | O
----------
O | X |  
----------
  |   | X
You loose.
